In [96]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt

In [97]:
train=pd.DataFrame(pd.read_csv('clean1Final.csv'))

In [98]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search


In [99]:
!pip install xgboost

In [101]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4


In [102]:
import time

In [103]:
target = 'mvar49'

In [104]:
train.keys().size

22

In [136]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgb_param['num_class']=4
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
             early_stopping_rounds=early_stopping_rounds,stratified=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    #print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob))
                    
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')



In [145]:
#Choose all predictors except target & IDcols
t=time.time()
predictors = [x for x in train.columns if x not in [target]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)

print('time',time.time()-t)


Model Report
Accuracy : 0.7862
time 59.483579874038696


In [168]:
df = pd.read_csv('leaderFinal2.csv')

In [162]:
df.keys()

Index(['Unnamed: 0', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6', 'mvar7',
       'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar13', 'mvar14', 'mvar15',
       'mvar16', 'mvar20', 'mvar24', 'mvar28', 'mvar32', 'mvar36', 'mvar40',
       'mvar41', 'mvar42', 'cm_key'],
      dtype='object')

In [172]:
del df['Unnamed: 0']

In [173]:
df.keys()

Index(['mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6', 'mvar7', 'mvar8', 'mvar9',
       'mvar10', 'mvar11', 'mvar13', 'mvar14', 'mvar15', 'mvar16', 'mvar20',
       'mvar24', 'mvar28', 'mvar32', 'mvar36', 'mvar40', 'mvar41', 'mvar42',
       'cm_key'],
      dtype='object')

In [175]:
del df['mvar3']

In [177]:
ids=df.pop('cm_key')

In [178]:
xgb1.predict_proba(df)



array([[ 0.73148692,  0.07840214,  0.12906398,  0.06104692],
       [ 0.84004384,  0.05951865,  0.08278651,  0.01765098],
       [ 0.64240271,  0.25653923,  0.00606078,  0.09499723],
       ..., 
       [ 0.616687  ,  0.06364144,  0.00300308,  0.31666854],
       [ 0.85041505,  0.03577263,  0.09381097,  0.02000134],
       [ 0.83204246,  0.07513637,  0.0657521 ,  0.0270691 ]], dtype=float32)

In [179]:
xgb1.predict_proba(df)



def answer(i):
    if(i == 1): return "Supp"
    elif(i == 2): return "Elite"
    elif(i == 3): return "Credit"

max_indices = np.argmax(xgb1.predict_proba(df)[:, 1:], axis=1)
max_vals = np.amax(xgb1.predict_proba(df)[:, 1:], axis=1)

y_final = 1 + np.argmax(xgb1.predict_proba(df)[:, 1:], axis=1)
y_final = y_final[np.argsort(max_vals)[::-1]]
class_final = [answer(i) for i in y_final]

#ids = ids[np.argsort(max_vals)]
ids = np.array(ids)[np.argsort(max_vals)[::-1]]

submission = pd.DataFrame({'c1': ids[:1000], 'c2': class_final[:1000]})


In [180]:
submission.to_csv("Top_Floors_IITGuwahati_6.csv", index = False, header = False)

In [181]:
np.unique(y_final, return_counts=True)

(array([1, 2, 3]), array([3770, 2877, 3353]))

In [182]:
t=time.time()
param_test1 = [{
 'max_depth':np.arange(4,6,1),
 'min_child_weight':np.arange(4,6,1)
}]

gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1,n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

print('time:',time.time()-t)

time: 1184.8907115459442


In [183]:
gsearch1.grid_scores_

[mean: 0.77887, std: 0.00080, params: {'max_depth': 3, 'min_child_weight': 1},
 mean: 0.77930, std: 0.00046, params: {'max_depth': 3, 'min_child_weight': 3},
 mean: 0.77925, std: 0.00059, params: {'max_depth': 3, 'min_child_weight': 5},
 mean: 0.77930, std: 0.00094, params: {'max_depth': 5, 'min_child_weight': 1},
 mean: 0.77920, std: 0.00086, params: {'max_depth': 5, 'min_child_weight': 3},
 mean: 0.77940, std: 0.00082, params: {'max_depth': 5, 'min_child_weight': 5},
 mean: 0.77722, std: 0.00146, params: {'max_depth': 7, 'min_child_weight': 1},
 mean: 0.77762, std: 0.00149, params: {'max_depth': 7, 'min_child_weight': 3},
 mean: 0.77805, std: 0.00075, params: {'max_depth': 7, 'min_child_weight': 5},
 mean: 0.77712, std: 0.00113, params: {'max_depth': 9, 'min_child_weight': 1},
 mean: 0.77724, std: 0.00079, params: {'max_depth': 9, 'min_child_weight': 3},
 mean: 0.77687, std: 0.00062, params: {'max_depth': 9, 'min_child_weight': 5}]

In [184]:
gsearch1.best_params_

{'max_depth': 5, 'min_child_weight': 5}

In [127]:
gsearch1.best_score_

-0.42362470244154915

In [82]:
df=pd.DataFrame(pd.read_csv('clean1Final.csv'))
y=df.pop('mvar49')

In [83]:
from sklearn.cross_validation import train_test_split
from sklearn import model_selection
xtrain, xtest, ytrain, ytest = train_test_split(df,y,test_size=0.3,stratify=y)

In [66]:
xtrain, xval, ytrain, yval = train_test_split(xtrain,ytrain,test_size=0.3,stratify=ytrain)

In [67]:
df.keys().size

21

In [74]:
from sklearn.ensemble import RandomForestClassifier
import time

t=time.time()

ranScore=0
nMax=2
mMax=2
ranFinal=RandomForestClassifier()

for m in range(1,10,1):
    for n  in range(10,130,10):
        ran=RandomForestClassifier(max_features=m,n_estimators=n,random_state=10).fit(xtrain,ytrain)
        temp=ran.score(xval,yval)
        #plt.scatter(n,temp,label=n)
        if(temp>ranScore):
            ranScore=temp
            nMax=n
            mMax=m
            ranFinal=ran
            print('cv score:',temp)
            #print('test Score', ranFinal.score(xtest,ytest))
    #plt.xlabel('no of trees')
    #plt.ylabel('no of features: '+str(m))
    #plt.legend()
    #plt.show()
    
print('Max n:',nMax)
print('Max m:',mMax)
print('train score:', ranFinal.score(xtrain,ytrain))
print('cv score:', ranFinal.score(xval,yval))
print('test Score', ranFinal.score(xtest,ytest))

print('time:', time.time()-t)

cv score: 0.772922636103
cv score: 0.777936962751
cv score: 0.77805635148
cv score: 0.778533906399
cv score: 0.778772683859
cv score: 0.779130850048
cv score: 0.779250238777
cv score: 0.779369627507
cv score: 0.779608404967
cv score: 0.779727793696
cv score: 0.779966571156
cv score: 0.780085959885
Max n: 80
Max m: 4
train score: 1.0
cv score: 0.780085959885
test Score 0.777601337234
time: 414.82312989234924


In [69]:
ranFinal.score(xtest,ytest)

0.77826995403259502

In [70]:
ranFinal.feature_importances_

array([ 0.03078782,  0.04960259,  0.08445443,  0.07131025,  0.0761748 ,
        0.0435157 ,  0.06588855,  0.01311478,  0.08461555,  0.05861975,
        0.01018137,  0.01706909,  0.07228277,  0.07259696,  0.07205608,
        0.036057  ,  0.04094753,  0.08823134,  0.00851449,  0.00277868,
        0.00120045])

In [ ]:
ranFinal.pred_proba(xtest)

In [72]:
train.keys()

Index(['mvar2', 'mvar4', 'mvar5', 'mvar6', 'mvar7', 'mvar8', 'mvar9', 'mvar10',
       'mvar11', 'mvar13', 'mvar14', 'mvar15', 'mvar16', 'mvar20', 'mvar24',
       'mvar28', 'mvar32', 'mvar36', 'mvar40', 'mvar41', 'mvar42'],
      dtype='object')

In [73]:
ranran=ranFinal

In [86]:
#xtrain, xtest, ytrain, ytest = model_selection.train_test_split(pokemonDf,legends,test_size=0.2,random_state=100)

from sklearn.ensemble import RandomForestClassifier

t=time.time()

splits=10


cv = model_selection.StratifiedKFold(n_splits=splits, shuffle=True, random_state=1)



ranScore=0
nMax=0
mMax=0

ranCV=[]
tempRanCV=[]
error=0
scoreCVSmall=0
tempScoreCV=0
scoreCV=0
testScore=0
trainScore=0
dMax=0
splitsMax=0


for m in range(3,4,1):
    for n  in range(80,130,10):
        scoreCVSmall=0
        tempRanCV=[]
        for traincv, testcv in cv.split(xtrain,ytrain):
            ran=RandomForestClassifier(max_features=m,n_estimators=n,random_state=1).fit(xtrain.iloc[traincv],ytrain.iloc[traincv])
            scoreCVSmall+=ran.score(xtrain.iloc[testcv],ytrain.iloc[testcv])
            tempRanCV.append(ran)
        tempScoreCV=scoreCVSmall/cv.n_splits

        if(tempScoreCV>scoreCV):
            scoreCV=tempScoreCV
            nMax=n
            mMax=m
            ranCV=tempRanCV
            splitsMax=splits
            print('cv error', tempScoreCV)



print('Max n:',nMax)
print('Max m:',mMax)
#print('Max d:',dMax)
print(splits,'-fold CV')

print('time:', time.time()-t)

cv error 0.778271303994
cv error 0.778557798539
cv error 0.778557901193
Max n: 110
Max m: 3
10 -fold CV
time: 243.38183522224426


In [87]:
print('Max n:',nMax)
print('Max m:',mMax)
print('Max d:',dMax)
print(splits,'-fold CV')

trainScore=0
for r in ranCV:
    trainScore+=r.score(xtrain,ytrain)
trainScore/=splitsMax
print('train Score1', trainScore)

testScore=0
for r in ranCV:
    testScore+=r.score(xtest,ytest)
testScore/=splitsMax
print('test Score1', testScore)

###########################################

for r in ranCV:
    r.fit(xtrain,ytrain)

trainScore=0
for r in ranCV:
    trainScore+=r.score(xtrain,ytrain)
trainScore/=splitsMax
print('train Score2', trainScore)


testScore=0
for r in ranCV:
    testScore+=r.score(xtest,ytest)
testScore/=splitsMax
print('test Score2', testScore)

#print('time:', time.time()-t)



Max n: 110
Max m: 3
Max d: 0
10 -fold CV
train Score1 0.977855786797
test Score1 0.778403677392
train Score2 1.0
test Score2 0.778520685332
